<a href="https://colab.research.google.com/github/sokrypton/ColabBio/blob/main/notebooks/replacement_scan.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Replacement Scan

Purpose: Find how many amino acid replacements your protein can tolerate and still make same prediction. Note, this protocol runs in single-sequence only (NO MSA). The analysis is likely only useful for denovo designed proteins.

In [ ]:
#@title setup
%%time
import os
if not os.path.isdir("params"):
  # get code
  os.system("pip -q install git+https://github.com/sokrypton/ColabDesign.git")
  # for debugging
  os.system("ln -s /usr/local/lib/python3.*/dist-packages/colabdesign colabdesign")
  # download params
  os.system("mkdir params")
  os.system("apt-get install aria2 -qq")
  os.system("aria2c -q -x 16 https://storage.googleapis.com/alphafold/alphafold_params_2022-12-06.tar")
  os.system("tar -xf alphafold_params_2022-12-06.tar -C params")

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import os
from colabdesign import mk_af_model, clear_mem
from colabdesign.af.alphafold.common import residue_constants

from IPython.display import HTML
from google.colab import files
import numpy as np

import tqdm.notebook
TQDM_BAR_FORMAT = '{l_bar}{bar}| {n_fmt}/{total_fmt} [elapsed: {elapsed} remaining: {remaining}]'


def get_pdb(pdb_code=""):
  if pdb_code is None or pdb_code == "":
    upload_dict = files.upload()
    pdb_string = upload_dict[list(upload_dict.keys())[0]]
    with open("tmp.pdb","wb") as out: out.write(pdb_string)
    return "tmp.pdb"
  elif os.path.isfile(pdb_code):
    return pdb_code
  elif len(pdb_code) == 4:
    os.system(f"wget -qnc https://files.rcsb.org/view/{pdb_code}.pdb")
    return f"{pdb_code}.pdb"
  else:
    os.system(f"wget -qnc https://alphafold.ebi.ac.uk/files/AF-{pdb_code}-F1-model_v3.pdb")
    return f"AF-{pdb_code}-F1-model_v3.pdb"

clear_mem()
af_model = mk_af_model(protocol="fixbb")

In [ ]:
#@title run
#@markdown #### Input Options
pdb = "6MRR" #@param {type:"string"}
chain = "A" #@param {type:"string"}
#@markdown #### Model Options
num_recycles = 0 #@param ["0", "1", "2", "3"] {type:"raw"}
#@markdown #### Scan Options
num_tries = -1 #@param [-1,10,20] {type:"raw"}
#@markdown - number of mutations to try before accepting. (`-1` = greedy, try all).
mut_res = "A" # @param ["None", "A","R","N","D","C","Q","E","G","H","I","L","K","M","F","P","S","T","W","Y","V"]
#@markdown - which mutation to make (`None` = mask the position completely).
sco_type = "rmsd+dgram" # @param ["rmsd+dgram","rmsd","dgram","pae"]
#@markdown - criteria to select by (`dgram` = cce between true and predicted distogram)

def get_sco(aux, sco_type):
  if sco_type == "rmsd":
    loss = aux["losses"]["rmsd"]
  elif sco_type == "dgram":
    loss = aux["losses"]["dgram_cce"]
  else:
    loss = aux["losses"]["rmsd"] + aux["losses"]["dgram_cce"]
  return loss

MUT = -1 if mut_res == "None" else residue_constants.restype_order[mut_res]

pdb_filename = get_pdb(pdb)
af_model.prep_inputs(pdb_filename=pdb_filename, chain=chain)
WT = af_model._wt_aatype.copy()
NEW_SEQ = WT.copy()

af_model.predict(seq=NEW_SEQ, verbose=False, hard=False, num_recycles=num_recycles)
af_model._save_results(verbose=False)

RMSDS = [af_model.aux["losses"]["rmsd"]]
SEQS = [NEW_SEQ]
loss = get_sco(af_model.aux, sco_type)
print(f"{af_model._k} rmsd={RMSDS[-1]:.3f} sco={loss:.3f}")
af_model._k += 1

while sum(NEW_SEQ != MUT):
  pos = np.where(NEW_SEQ != MUT)[0]
  if num_tries > -1:
    pos = np.random.permutation(pos)[:num_tries]
  buff = []
  losses = []
  for t in pos:
    test_seq = NEW_SEQ.copy()
    test_seq[t] = MUT
    aux = af_model.predict(seq=test_seq, return_aux=True, verbose=False, hard=False, num_recycles=num_recycles)
    buff.append({"aux":aux, "seq":test_seq})
    losses.append(get_sco(aux, sco_type))

  # accept best
  best_idx = np.argmin(losses)
  best = buff[best_idx]
  NEW_SEQ = best["seq"]
  RMSDS.append(best["aux"]["losses"]["rmsd"])
  SEQS.append(NEW_SEQ)

  af_model.aux = best["aux"]
  af_model.set_seq(seq=NEW_SEQ)
  af_model._save_results(verbose=False)
  print(f"{af_model._k} rmsd={RMSDS[-1]:.3f} sco={losses[best_idx]:.3f}")
  af_model._k += 1

In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize=(5,4), dpi=100)
plt.plot([sum(seq == MUT)/len(seq) for seq in SEQS], RMSDS)
plt.yscale("log")
plt.yticks([0.5,1,2,4,8,16,32],[0.5,1,2,4,8,16,32])
plt.xlim([0,1])
plt.xlabel(f"fraction of '{mut_res}'")
plt.ylabel("rmsd")
plt.show()

In [ ]:
HTML(af_model.animate(dpi=100))